### Decorators 2

> Aaron's Experiments on Decorator 2

## <font color=palevioletred> Aaron的Title
###  **Python 裝飾器進階：如何讓裝飾器支援參數（以計時器為例）**  

### **Summary:**  
一般的 Python 裝飾器只能裝飾函式，無法直接傳入額外參數。為了解決這個問題，我們可以使用「裝飾器工廠」，讓一個函式回傳真正的裝飾器。這樣我們就能靈活設定裝飾器的行為，例如控制函式執行次數來計算平均執行時間。本篇文章以計時器為例，說明如何使用這種技巧，並探討其應用場景，如日誌記錄、權限管理與重試機制，讓 Python 程式更具彈性與可讀性。

## Aaron 的本 Notebook 的重點摘要：

### 總結 Jupyter Notebook 內容：Python 裝飾器（Decorators）

這份 Notebook 主要介紹 **Python 的裝飾器（Decorators）**，並涵蓋了以下幾個關鍵概念：

---

### **1. 基本的裝飾器**
裝飾器是一個函數，它可以在不修改原始函數的情況下，擴展該函數的功能。例如，`timed` 裝飾器計算函數的執行時間：

```python
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print(f'Run time: {elapsed:.6f}s')
        return result

    return inner
```

可以用這個裝飾器來測量 Fibonacci 函數的執行時間：
```python
@timed
def fib(n):
    return 1 if n < 3 else fib(n-1) + fib(n-2)
```

---

### **2. 帶參數的裝飾器**
普通的裝飾器只能裝飾函數，而無法接收額外的參數。例如，我們希望 `timed` 裝飾器可以指定執行的次數並計算平均執行時間：

```python
def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for _ in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_elapsed = total_elapsed / num_reps
            print(f'Avg Run time: {avg_elapsed:.6f}s ({num_reps} reps)')
            return result
        return inner
    return decorator
```
使用方式：
```python
@timed(5)  # 這裡傳入 5，表示函數將執行 5 次，計算平均運行時間
def fib(n):
    return 1 if n < 3 else fib(n-1) + fib(n-2)
```

---

### **3. 裝飾器工廠（Decorator Factory）**
為了實現帶參數的裝飾器，我們需要使用「裝飾器工廠」，即 **返回裝飾器的函數**。

```python
def dec_factory(msg):
    def decorator(fn):
        def inner(*args, **kwargs):
            print(f'{msg}: Running function {fn.__name__}')
            return fn(*args, **kwargs)
        return inner
    return decorator
```
用法：
```python
@dec_factory("DEBUG")
def my_func():
    print("Executing my_func")

my_func()
```
輸出：
```
DEBUG: Running function my_func
Executing my_func
```

---

### **4. `functools.wraps` 修飾內部函數**
當我們使用裝飾器時，會影響被裝飾函數的 `__name__` 和 `__doc__` 屬性。例如：
```python
@timed(5)
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

print(fib.__name__)  # 這裡會顯示 'inner' 而不是 'fib'
```
為了保持原函數的元數據（metadata），我們可以使用 `functools.wraps`：

```python
from functools import wraps

def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter
        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for _ in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_elapsed = total_elapsed / num_reps
            print(f'Avg Run time: {avg_elapsed:.6f}s ({num_reps} reps)')
            return result
        return inner
    return decorator
```
這樣，`fib.__name__` 仍然會是 `'fib'`。

---

### **5. Python 裝飾器的語法解析**
當我們寫：
```python
@decorator
def my_func():
    pass
```
Python 會自動執行：
```python
my_func = decorator(my_func)
```

如果 `@decorator` 需要參數，例如 `@decorator(10)`，這時候 `decorator(10)` **應該返回一個新的裝飾器**，然後這個新裝飾器再用來裝飾 `my_func`。

```python
def decorator_factory(param):
    def decorator(fn):
        def inner(*args, **kwargs):
            print(f"Decorator parameter: {param}")
            return fn(*args, **kwargs)
        return inner
    return decorator
```
使用方式：
```python
@decorator_factory(10)
def my_func():
    print("Hello")

my_func()
```
輸出：
```
Decorator parameter: 10
Hello
```

---

### **總結**
1. **基本裝飾器** 可以修改函數的行為，例如計時器裝飾器 `timed`。
2. **帶參數的裝飾器** 需要使用「裝飾器工廠」，即 **返回裝飾器的函數**。
3. **使用 `functools.wraps` 保持原函數的元數據**，否則 `__name__` 會變成內部函數 `inner`。
4. **裝飾器的底層運作方式** 是 `my_func = decorator(my_func)`，如果 `@decorator(10)`，那麼 `decorator(10)` 需要返回一個裝飾器，然後再應用到函數上。

這份 Notebook 主要讓你理解 Python **裝飾器的基礎與進階用法**，透過多種示例來展示如何設計高效能的裝飾器！

## <font color=plum>提綱內容 --Aaron

### **我們已經學會如何創建簡單與稍微複雜的裝飾器**

不過，我們也一直在使用一些內建的裝飾器（decorators），例如 `wraps` 和 `lru_cache`，這些裝飾器可以接受參數。

這其實非常有用，而我們也可以自己實作相同的功能。

---

### **使用 `@` 語法或較長的語法來裝飾費氏數列函式**

我們可以使用 `@` 語法來裝飾 Fibonacci 函式，或者使用較長的語法，如下所示：

```python
@some_decorator
def fibonacci(n):
    ...
```

或者：

```python
fibonacci = some_decorator(fibonacci)
```

---

### **修改計時器讓函式執行多次，並計算平均執行時間**

我們來改寫計時裝飾器，使它能夠讓函式執行多次，然後計算出平均的執行時間：

```python
import time

def timed(fn):
    def inner(*args, **kwargs):
        total_time = 0
        for _ in range(10):  # 執行 10 次
            start = time.perf_counter()
            result = fn(*args, **kwargs)
            end = time.perf_counter()
            total_time += (end - start)
        avg_time = total_time / 10
        print(f"Average execution time: {avg_time:.5f} seconds")
        return result
    return inner
```

---

### **讓執行次數成為參數，而不是寫死的數字**

目前我們讓函式執行 10 次來計算平均時間，但這個 `10` 是寫死的（hardcoded）。我們應該讓它變成一個參數，這樣使用者可以自己決定要執行幾次。

---

### **當裝飾器需要參數時，無法直接使用 `@` 語法**

當我們希望 `timed` 裝飾器接受參數時，不能直接使用 `@` 語法，因為在 `@decorator` 這種語法下，Python **只會將被裝飾的函式作為參數傳入裝飾器**，而不允許我們再額外傳入其他參數。

例如：

```python
@timed(10)  # 這樣是不行的，因為 `@timed` 期望一個函式，而不是一個數字
def fibonacci(n):
    ...
```

要解決這個問題，我們需要讓 `timed` 本身變成一個「裝飾器工廠」（decorator factory），這樣我們可以先傳入參數，然後再返回真正的裝飾器。

---

### **Python 的裝飾器其實是如何運作的？**

當我們使用裝飾器時，例如：

```python
@dec
def my_func():
    pass
```

這其實等同於：

```python
my_func = dec(my_func)
```

換句話說，當我們裝飾 `my_func` 時，Python 會「**調用**」`dec`，並將 `my_func` 作為參數傳入。

那麼，如果 `dec` 本身不是裝飾器，而是 **一個回傳裝飾器的函式** 呢？

---

### **讓裝飾器工廠回傳裝飾器**

我們可以改寫 `timed`，讓它變成一個函式，回傳一個真正的裝飾器：

```python
import time

def timed_factory(num_reps):
    def timed(fn):
        def inner(*args, **kwargs):
            total_time = 0
            for _ in range(num_reps):  # 使用者可以決定執行次數
                start = time.perf_counter()
                result = fn(*args, **kwargs)
                end = time.perf_counter()
                total_time += (end - start)
            avg_time = total_time / num_reps
            print(f"Average execution time over {num_reps} runs: {avg_time:.5f} seconds")
            return result
        return inner
    return timed  # 回傳裝飾器
```

使用方式：

```python
@timed_factory(10)  # 這樣就可以使用 `@` 了！
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

這相當於：

```python
fibonacci = timed_factory(10)(fibonacci)
```

---

### **關鍵概念：我們先「呼叫」裝飾器工廠，再使用它回傳的裝飾器**
請注意，這裡的關鍵是我們 **先呼叫 `timed_factory(10)`**，然後才使用它的回傳值（即裝飾器 `timed`）來裝飾函式。

由於我們 **可以在 `timed_factory` 內部存取 `num_reps`**，這樣我們就能在內部函式中使用這個值，而不影響裝飾器的結構。

---

### **最後的調整：讓裝飾器名稱更簡潔**
我們不太希望裝飾器工廠叫做 `timed_factory`，而是希望它就叫 `timed`，這樣使用起來會更直覺：

```python
def timed(num_reps):  # 直接叫 `timed`
    def decorator(fn):
        def inner(*args, **kwargs):
            total_time = 0
            for _ in range(num_reps):
                start = time.perf_counter()
                result = fn(*args, **kwargs)
                end = time.perf_counter()
                total_time += (end - start)
            avg_time = total_time / num_reps
            print(f"Average execution time over {num_reps} runs: {avg_time:.5f} seconds")
            return result
        return inner
    return decorator
```

使用方式：

```python
@timed(10)
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

---

### **結論**
1. **裝飾器的 `@` 語法只能傳入一個函式**，如果我們想傳入額外的參數，就必須使用「**裝飾器工廠**」來回傳一個真正的裝飾器。
2. **透過嵌套函式**，我們可以存取外層函式的變數，讓裝飾器可以帶參數。
3. **這種技巧適用於很多場景**，例如記錄日誌（logging）、權限控制（authentication）、重試機制（retrying）等。

現在，我們成功讓 `timed` 可以接受參數，並透過 `@timed(10)` 的語法來計時 Fibonacci 函式的執行時間！🔥🚀

## 說人話

### **讓 Python 裝飾器支援參數：計時範例**  

---

### **1. 為什麼要讓裝飾器支援參數？**  
我們之前學過裝飾器，例如這個計時器：

```python
import time

def timed(fn):
    def inner(*args, **kwargs):
        start = time.perf_counter()
        result = fn(*args, **kwargs)
        end = time.perf_counter()
        print(f"Execution time: {end - start:.5f} seconds")
        return result
    return inner
```

這樣的裝飾器可以用來測試函式的執行時間：

```python
@timed
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

但這樣有個問題：**我們只能測一次！**  
如果我們想 **執行多次並計算平均時間**，就得手動改程式碼，這不太靈活。

---

### **2. 讓裝飾器可以設定執行次數**
我們希望這樣使用：

```python
@timed(10)  # 讓函式執行 10 次，計算平均時間
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

但這樣會出錯！因為 Python 預設 `@decorator` **只能傳入函式本身**，不能再額外傳參數。

---

### **3. 解法：用「裝飾器工廠」來產生裝飾器**
我們的解法是「**先呼叫一個函式來生成裝飾器**」，這樣我們就能在裝飾器裡存取參數：

```python
import time

def timed(num_reps):  # 讓使用者決定執行次數
    def decorator(fn):
        def inner(*args, **kwargs):
            total_time = 0
            for _ in range(num_reps):  # 執行 num_reps 次
                start = time.perf_counter()
                result = fn(*args, **kwargs)
                end = time.perf_counter()
                total_time += (end - start)
            avg_time = total_time / num_reps
            print(f"Average execution time over {num_reps} runs: {avg_time:.5f} seconds")
            return result
        return inner
    return decorator  # 回傳真正的裝飾器
```

現在我們可以這樣用：

```python
@timed(10)  # 讓 Fibonacci 執行 10 次，計算平均時間
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
```

Python 會執行 `timed(10)` 來回傳一個裝飾器，然後再用這個裝飾器來裝飾 `fibonacci`。

---

### **4. 這樣做的好處**
✅ **讓裝飾器變靈活**：可以自由設定執行次數，而不是寫死在程式裡。  
✅ **保持 `@` 裝飾器語法**：不需要手動包裝函式，讓程式碼更乾淨。  
✅ **可以應用到更多場景**：除了計時，也可以用在日誌記錄、權限控制等。

---

### **5. 總結**
1. **一般的裝飾器** 只能裝飾函式，不能傳額外參數。
2. **我們可以用「函式回傳裝飾器」的方式** 來解決這個問題。
3. **這種技巧很常用**，不只適用於計時，也可以用來做 **重試機制、日誌記錄、權限管理** 等等。

這樣講比較白話吧？ 😆

We have seen how to create some simple and not so simple decorators.

However we have also been using built-in decorators that can accept parameters, such as `wraps` and `lru_cache`.

This can be quite useful and we can accomplish the same thing ourselves.

First recall our original timer decorator from an earlier video (Decorator Application - Timer):

In [1]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print('Run time: {0:.6f}s'.format(elapsed))
        return result
    
    return inner

In [2]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-1) + calc_fib_recurse(n-2)

def fib(n):
    return calc_fib_recurse(n)

We can decorate our Fibonacci function using the **@** syntax, or the longer syntax as follows:

In [3]:
fib = timed(fib)

In [4]:
fib(30)

Run time: 0.255260s


832040

Let's modify this so the timer runs the function multiple times and calculates the average run time:

In [5]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / 10
        print('Avg Run time: {0:.6f}s'.format(avg_elapsed))
        return result
    
    return inner

And again we decorate it using the long syntax:

In [6]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib)

In [7]:
fib(28)

Avg Run time: 0.098860s


317811

But that value of 10 has been hardcoded. Let's make it a parameter instead.

In [8]:
def timed(fn, num_reps):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result
    
    return inner

Now to decorate our Fibonacci function we **have** to use the long syntax (as we saw in the lecture, the **@** syntax will not work):

In [9]:
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib, 5)

In [10]:
fib(28)

Avg Run time: 0.095708s (5 reps)


317811

The problem is that we cannot use the `@` decorator syntax because when using that syntax Python passes a **single** argument to the decorator: the function we are decorating - nothing else.

Of course we could just use what we did above, but the decorator syntax is kind of neat, so it would be nice to retain the ability to use it.

We just need to change our thinking a little bit to do this:

First, when we see the following syntax:

`
@dec
def my_func():
    pass
`

we see that `dec` must be a function that takes a single argument, the function being decorated.

You'll note that `dec` is just a function, but we do not **call** `dec` when we decorate `my_func`, we simply use the label `dec`.

Then Python does:

`
my_func = dec(my_func)
`

Let's try a concrete example:

In [11]:
def dec(fn):
    print ("running dec")
    
    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)
              
    return inner

In [12]:
@dec
def my_func():
    print('running my_func')

running dec


As we can see, when we decorated `my_func`, the `dec` function was **called** at that time.

(Because Python did this: 

`my_func = dec(my_func)` 

so `dec` was called)

And when we now call `my_func`, we see that the `inner` function is called, followed by the original `my_func`

In [13]:
my_func()

running inner
running my_func


But what if `dec` was not the decorator itself, but instead created and returned a decorator?

Let's see how we might do this:

In [14]:
def dec_factory():
    print('running dec_factory')
    def dec(fn):
        print('running dec')
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
        return inner
    return dec

So as you can see, calling `dec_generator()` will return that `dec` function which is our decorator:

In [15]:
@dec_factory()
def my_func(a, b):
    print(a, b)

running dec_factory
running dec


You can see that both `dec_generator` and `dec` were already called.

In [16]:
my_func(10, 20)

running inner
10 20


And there you go, all we did is basically create a decorator by calling a function (`dec_factory`) and use the return value of that call (the `dec` function) as our actual decorator.

We could have done the decoration this way too:

In [17]:
dec = dec_factory()

running dec_factory


In [18]:
@dec
def my_func():
    print('running my_func')

running dec


In [19]:
my_func()

running inner
running my_func


Or even this way:

In [20]:
dec = dec_factory()

def my_func():
    print('running my_func')

my_func = dec(my_func)

running dec_factory
running dec


In [21]:
my_func()

running inner
running my_func


Of course we could even decorate it this way using a single statement:

In [22]:
def my_func():
    print('running my_func')

my_func = dec_factory()(my_func)

running dec_factory
running dec


In [23]:
my_func()

running inner
running my_func


OK, so now we have decorated our function using, not a decorator, but a decorator factory as follows:

In [24]:
def dec_factory():
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            return fn(*args, **kwargs)
        return inner
    return dec

In [25]:
@dec_factory()
def my_func(a, b):
    return a + b

In [26]:
my_func(10, 20)

running decorator inner


30

You should note that in this approach, we are **calling** `dec_factory()`, [note the parentheses `()`], and **then** using the return value (a decorator) to decorate our function.

So, we could pass arguments as we do so without affecting the final outcome. In fact we can even access them from anywhere inside `dec_factory`, including any of the nested functions! 

Let's try this:

In [27]:
def dec_factory(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('running decorator inner')
            print('free vars: ', a, b)  # a and b are free variables!
            return fn(*args, **kwargs)
        return inner
    return dec

In [28]:
@dec_factory(10, 20)
def my_func():
    print('python rocks')

In [29]:
my_func()

running decorator inner
free vars:  10 20
python rocks


And this is how we can create decorators with parameters. We do not directly create a decorator, instead we use an outer function that returns a decorator when called, and pass arguments to that outer function, which the decorator and its inner function can of course access as nonlocal (free) variables.

So now, let's go back to our original problem where we wanted our timing decorator to run a number of loops which could be specified as a parameter when decorating the function we want to time.

Here it is again:

In [30]:
def timed(fn, num_reps):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result
    
    return inner

So, all we need to do is create an outer function around our timed decorator, and pass the `num_reps` argument to that outer function instead:

In [31]:
def timed_factory(num_reps=1):
    def timed(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return timed    

In [32]:
@timed_factory(5)
def fib(n):
    return calc_fib_recurse(n)

In [33]:
fib(30)

Avg Run time: 0.249934s (5 reps)


832040

Just to put the finishing touch on this, we probably don't want to have our outer function named the way it is (`timed_factory`). Instead we probably just want to call it `timed`. So lets just do this final part:

In [34]:
from functools import wraps

def timed(num_reps=1):
    def decorator(fn):
        from time import perf_counter

        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(num_reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / num_reps
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                            num_reps))
            return result
        return inner
    return decorator  

In [35]:
@timed(5)
def fib(n):
    return calc_fib_recurse(n)

In [36]:
fib(30)

Avg Run time: 0.253744s (5 reps)


832040